In [ ]:
#Importing original dataset via CSV (https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset)
import pandas as pd
def load_dataset(csv):
  df = pd.read_csv('online_shoppers_intention.csv')
  return df
csv = 'online_shoppers_intention.csv'
df = load_dataset(csv)

#Feature Engineering
#Converting Categorical into Integers and Use One-Hot Coding
sessions = df
sessions['Revenue'] = sessions['Revenue'].astype(int)#converting revenue into 0 and 1
sessions['SpecialDay'] = sessions['SpecialDay'].astype(int)
# One-hot encode VisitorType
visitor_type_dummies = pd.get_dummies(sessions['VisitorType'], prefix='VisitorType').astype(int)
sessions = pd.concat([sessions, visitor_type_dummies], axis=1)
sessions.drop('VisitorType', axis=1, inplace=True)
sessions['Weekend']=sessions['Weekend'].astype(int)
sessions['Month'] = sessions['Month'].map({'Feb':2, 'Mar':3,'May':5, 'June':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12})

#Adding Relevant Columns
sessions['SessionDuration']=sessions['Administrative_Duration']+sessions['Informational_Duration']+sessions['ProductRelated_Duration']
sessions['TotalPagesViewed']=sessions['Administrative']+sessions['Informational']+sessions['ProductRelated']
sessions['BounceRates'] = sessions['BounceRates'].astype(float)
sessions.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
import scipy.stats as stats


In [ ]:
#How many K Means we should use with Elbow Method

def elbow_method(data, max_k):
  inertias = []
  for i in range(1, max_k):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)
  plt.plot(range(1, max_k), inertias, 'o-')
  plt.title('Elbow Method')
  plt.xlabel('Number of clusters')
  plt.ylabel('Inertia')
  plt.grid(True)
  plt.show()

elbow_method(sessions, 15)

In [ ]:
def random_centroids(data, k):
    centroids = []
    for i in range(k):
        centroid = data.apply(lambda x: float(x.sample()))
        centroids.append(centroid)
    return pd.concat(centroids, axis=1)

def get_labels(data, centroids):
    distances = centroids.apply(lambda x: np.sqrt(((data - x) ** 2).sum(axis=1)))
    return distances.idxmin(axis=1)

def new_centroids(data, labels, k):
    centroids = data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T
    return centroids

def plot_clusters(data, labels, centroids, iteration):
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)
    centroids_2d = pca.transform(centroids.T)
    clear_output(wait=True)
    plt.title(f'Iteration {iteration}')
    plt.scatter(x=data_2d[:,0], y=data_2d[:,1], c=labels)
    plt.scatter(x=centroids_2d[:,0], y=centroids_2d[:,1])
    plt.show()

In [178]:
value_of_k = 6
centroids = random_centroids(sessions, value_of_k)
labels = get_labels(sessions, centroids)
labels.value_counts()

C:\Users\Owner\AppData\Local\Temp\ipykernel_12780\748041225.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  centroid = data.apply(lambda x: float(x.sample()))


4    5035
5    3985
2    2737
1     497
0      76
Name: count, dtype: int64

In [ ]:
max_iterations = 100
centroid_count = value_of_k

centroids = random_centroids(sessions, centroid_count)
old_centroids = pd.DataFrame()
iteration = 1

while iteration < max_iterations and not centroids.equals(old_centroids):
    old_centroids = centroids
    
    labels = get_labels(sessions, centroids)
    centroids = new_centroids(sessions, labels, centroid_count)
    plot_clusters(sessions, labels, centroids, iteration)
    iteration += 1

In [ ]:
centroids

In [ ]:
import scipy.stats as stats

# Initialize an empty dictionary to store results
results = {}

# Loop through each feature in the sessions DataFrame
for feature in sessions.columns:
    if feature != 'labels':
        # Perform ANOVA for the current feature across all centroids
        p_value = stats.f_oneway(*[sessions[sessions['labels'] == i][feature] for i in range(value_of_k)])[1]
        # Store the p-value in the results dictionary
        results[feature] = p_value

# Initialize an empty dictionary to store significant features
significant_features = {}

# Loop through the results dictionary
for feature, p_value in results.items():
    # Check if the p-value is less than 0.05
    if p_value < 0.05:
        # Add the feature to the significant_features dictionary
        significant_features[feature] = p_value

# Print the significant features
print("Significant Features:", significant_features)

C:\Users\Owner\AppData\Roaming\Python\Python312\site-packages\scipy\stats\_stats_py.py:4102: DegenerateDataWarning: at least one input has length 0
  if _f_oneway_is_too_small(samples):


Significant Features: {}


In [195]:
import scipy.stats as stats

# Initialize an empty dictionary to store p-values for each feature
results = {}

# Loop through each feature in the DataFrame, excluding the 'labels' column
for feature in sessions.columns:
    if feature != 'labels':
        # Gather data for the feature across all non-empty clusters
        feature_data = [sessions[labels == i][feature] for i in range(value_of_k) if len(sessions[labels == i]) > 0]
        
        # Ensure there are enough samples per group for ANOVA (at least two non-empty groups)
        if all(len(data) > 1 for data in feature_data) and len(feature_data) > 1:
            # Perform ANOVA for the current feature across all centroids
            stat, p_value = stats.f_oneway(*feature_data)
            # Store the p-value in the results dictionary
            results[feature] = p_value
        else:
            print(f"Not enough data for ANOVA in feature '{feature}'")

# Initialize a dictionary to store only the significant features
significant_features = {feature: p_value for feature, p_value in results.items() if p_value < 0.05}

# Print the significant features
print("Significant Features:", significant_features)


Significant Features: {'Administrative': 0.0, 'Administrative_Duration': 0.0, 'Informational': 9.758683011124056e-303, 'Informational_Duration': 0.0, 'ProductRelated': 0.0, 'ProductRelated_Duration': 0.0, 'BounceRates': 0.0, 'ExitRates': 0.0, 'PageValues': 2.0786277299896437e-69, 'SpecialDay': 0.018612362598686098, 'Month': 7.968736520034775e-48, 'TrafficType': 2.9129139811946957e-08, 'Weekend': 0.0022962125580584794, 'Revenue': 2.850819539483301e-113, 'VisitorType_New_Visitor': 3.22111047471983e-50, 'VisitorType_Other': 1.1997513483115759e-05, 'VisitorType_Returning_Visitor': 1.4068193938878273e-53, 'SessionDuration': 0.0, 'TotalPagesViewed': 0.0}


In [ ]:
#Identify individual sessions in the columns 
features=['Revenue', 'SessionDuration', 'TotalPagesViewed', 'BounceRates', 'ExitRates', 'PageValues', 'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'Weekend', 'VisitorType_New_Visitor', 'VisitorType_Other', 'VisitorType_Returning_Visitor']
#features=['Revenue', 'SessionDuration', 'TotalPagesViewed', 'BounceRates', 'ExitRates', 'PageValues', 'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'Weekend', 'VisitorType']

#View the values in each Cluster if meaningful*
print(sessions[labels==1]['PageValues'])

39       0.0
75       0.0
117      0.0
147      0.0
160      0.0
        ... 
12241    0.0
12254    0.0
12298    0.0
12324    0.0
12326    0.0
Name: PageValues, Length: 497, dtype: float64

In [182]:
kmeans = KMeans(6)
kmeans.fit(sessions)
sk_centroids=kmeans.cluster_centers_

In [184]:
sk_labels = get_labels(sessions, pd.DataFrame(sk_centroids).T)

In [ ]:
# Initialize an empty dictionary to store results
results = {}

# Get the labels from the kmeans object
kmeans_labels = kmeans.labels_

# Loop through each feature in the sessions DataFrame
for feature in sessions.columns:
    if feature != 'labels':
        # Perform ANOVA for the current feature across all centroids
        p_value = stats.f_oneway(*[sessions[kmeans_labels == i][feature] for i in range(value_of_k)])[1]
        # Store the p-value in the results dictionary
        results[feature] = p_value

# Initialize an empty dictionary to store significant features
significant_features = {}

# Loop through the results dictionary
for feature, p_value in results.items():
    # Check if the p-value is less than 0.05
    if p_value < 0.05:
        # Add the feature to the significant_features dictionary
        significant_features[feature] = p_value

# Print the significant features
print("Significant Features:", significant_features)

Significant Features: {'Administrative': 0.0, 'Administrative_Duration': 0.0, 'Informational': 0.0, 'Informational_Duration': 0.0, 'ProductRelated': 0.0, 'ProductRelated_Duration': 0.0, 'BounceRates': 1.7872193668700428e-133, 'ExitRates': 1.4684121613067783e-274, 'PageValues': 1.0594980807316606e-35, 'Month': 1.3683604593730615e-59, 'Region': 0.003979309260472367, 'TrafficType': 1.1403300340276675e-05, 'Revenue': 1.22952009743693e-81, 'VisitorType_New_Visitor': 2.7835625267847577e-54, 'VisitorType_Other': 0.016681381413509125, 'VisitorType_Returning_Visitor': 7.211993899974316e-58, 'SessionDuration': 0.0, 'TotalPagesViewed': 0.0}
